# Prepare Inventory

In [4]:
# from esdc_inventory import read_esdc_inventory
# import glob

# resp_file_path1 = r"/Users/jygong/Stanford/Research_Data/XZ_data_from_Jiangtao/*MSEED/resp/*LH*"
# resp_file_path2 = r"/Users/jygong/Stanford/Research_Data/XZSeis_250209NewData/*/resp/*LH*"
# resp_file_list = glob.glob(resp_file_path1)
# resp_file_list.extend(glob.glob(resp_file_path2))
# resp_meta_file = r"./Inputs/XZ_LH.stations"

# inventory = read_esdc_inventory(resp_meta_file,resp_file_list)

# output_file = "./Inputs/XZ_LH_inventory1.xml"
# inventory.write(output_file, format="STATIONXML")

# Prepare Catalog

In [5]:
import pandas as pd
from interpolate_moho import get_moho, load_gravity_moho_grid, load_rf_moho_grid

# ───── CONFIGURATION ─────
input_file = "Tibet.csv"
output_file = "../Inputs/Tibet.csv"
gravity_mohofile = "./Moho_Files/GravityMoho.txt"
rf_mohofile = "./Moho_Files/Moho_RF_int.txt"
apply_depth_filter = True  # Set to False to skip filtering
depth_threshold = 0       # Minimum depth (in km) if filtering is applied
# ─────────────────────────

# ───── Load and rename columns ─────
df = pd.read_csv(input_file)
df.rename(columns={
    'time': 'event_time',
    'latitude': 'event_lat',
    'longitude': 'event_lon',
    'depth': 'event_dep',
    'magnitude': 'event_mag',
}, inplace=True)

# ───── Subset necessary columns ─────
columns_to_keep = [
    'event_time', 'event_mag', 'event_lat', 'event_lon', 'event_dep',
]
df_subset = df[columns_to_keep].copy()

# ───── Optional depth filter ─────
if apply_depth_filter:
    df_subset = df_subset[df_subset['event_dep'] > depth_threshold]

print(f"Number of events: {len(df_subset)}")

# ───── Interpolate Moho depth ─────
for idx, event in df_subset.iterrows():
    evlo = event['event_lon']
    evla = event['event_lat']
    moho, _ = get_moho(filepath=gravity_mohofile, evlo=evlo, evla=evla, 
                       load_moho_grid_func=load_gravity_moho_grid, is_plot=False)
    df_subset.loc[idx, 'Moho_gravity'] = round(moho,1)  # Round to 1 decimal place
    moho_1, _ = get_moho(filepath=rf_mohofile, evlo=evlo, evla=evla, 
                       load_moho_grid_func=load_rf_moho_grid, is_plot=False)
    df_subset.loc[idx, 'Moho_rf'] = round(moho_1,1)  # Round to 1 decimal place
    print(idx, round(moho,1),round(moho_1,1))

# ───── Save output ─────
df_subset.to_csv(output_file, index=False)


Number of events: 10
0 47.3 56.5
1 64.1 50.7
2 48.3 44.9
3 54.0 49.9
4 46.7 45.0
5 43.9 45.8
6 40.4 42.5
7 47.7 54.0
8 39.1 41.2
9 44.6 42.3


In [3]:
# ######## Code that may be useful #########
# # Filter the catalog according to time

# import pandas as pd

# # ───── Configuration ─────
# input_file = 'XZ_NEIC0_TP.csv'
# output_file = 'XZ_NEIC1_TP.csv'
# cutoff_date = pd.Timestamp('2009-01-01', tz='UTC')  # UTC-aware cutoff
# # ─────────────────────────

# # ───── Load and preprocess ─────
# cata_df = pd.read_csv(input_file)

# # Convert to UTC-aware datetime
# cata_df['event_time'] = pd.to_datetime(cata_df['event_time'], utc=True)

# # ───── Apply time filter ─────
# filtered_df = cata_df[cata_df['event_time'] >= cutoff_date].copy()

# # Format to ISO8601 with millisecond precision and trailing 'Z'
# filtered_df['event_time'] = (
#     filtered_df['event_time']
#     .dt.strftime('%Y-%m-%dT%H:%M:%S.%f')
#     .str.slice(0, 23) + 'Z'
# )

# # ───── Output ─────
# print(f"Found {len(filtered_df)} events after {cutoff_date.date()}")
# print(filtered_df)

# filtered_df.to_csv(output_file, index=False)
